In [ ]:
import sqlite3
from urllib.parse import urlparse
import os
import re
import json

DB_PATH = './teledeck.db'

conn = sqlite3.connect(DB_PATH)


cursor = conn.cursor()
cursor2 = conn.cursor()
url_pattern = r'https?://[^\s]+'

def extract_links_and_domains(text):
    links = re.findall(url_pattern, text)
    domains = [urlparse(link).netloc for link in links]
    return links, domains


all_links = []
all_domains = set()

for row in cursor.execute("SELECT text FROM media_items where text <> ''"):
    links, domains = extract_links_and_domains(row[0])
    if links:
        all_links.append(links)
        all_domains.update(domains)

print("\nUnique domain names:")
for domain in all_domains:
    print(domain)


json.dump({'links': all_links, 'domains': list(domains)}, open('data/links.json', 'w'))

In [ ]:
flat_links = sorted(list(set([link for links in all_links for link in links])))
len(flat_links)

In [ ]:
[print(domain) for domain in all_domains]


In [ ]:

# Treat these separately "t.co"

import re
twitter_alikes = [
    "twitter",
    "pxtwitter",
    "vxtwitter",
    "zztwitter",
    "fixupx",
    "vxTwitter",
    "twxtter",
    "fixvx",
    "FXTwitter"
]

twitter_alikes_re = '|'.join(twitter_alikes)
is_twitter_alike = re.compile(twitter_alikes_re)

twitter_links = [link for link in flat_links if is_twitter_alike.search(link.lower())]
len(twitter_links)

In [ ]:
extract_user = re.compile(r'https?://[^/]+/([^/]+)/(.+)$')
extract_alternate = re.compile(f'https?://(www\.)?({twitter_alikes_re})(.com|.co)?/([^/\)\.\?]+)')


users = set()
for link in twitter_links:
    match = extract_user.match(link)
    if match:
        users.add(match[1])
        continue
    match = extract_alternate.match(link)
    if match:
        users.add(match[1])
    else:
        print(link)


users = list(users)
print(users)

In [ ]:
json.dump(users, open('data/twitter_users.json', 'w'))